In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
keras.backend.set_image_data_format("channels_last")
import tensorflow as tf

import numpy as np
import random

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

2025-07-12 21:00:52.224060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752346852.242760 3052685 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752346852.250579 3052685 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752346852.268346 3052685 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752346852.268381 3052685 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752346852.268384 3052685 computation_placer.cc:177] computation placer alr

In [2]:
try:
    os.chdir("/home/das214/PQuant/mdmm_dev/src")
except:
    pass

for f in os.listdir(os.getcwd()):
    print(f)

pquant
smartpixels


In [3]:
dataset_base_dir = "/depot/cms/users/das214/datasets/dataset_3sr/dataset_3sr_16x16_50x12P5_parquets/contained"
tfrecords_base_dir = os.path.join(dataset_base_dir, "TFR_files", "2t")

dataset_train_dir = os.path.join(dataset_base_dir, "train")
dataset_test_dir = os.path.join(dataset_base_dir, "test")
tfrecords_dir_train = os.path.join(tfrecords_base_dir, "TFR_train")
tfrecords_dir_val   = os.path.join(tfrecords_base_dir, "TFR_val")

train_file_size = len(os.listdir(dataset_train_dir))
val_file_size = len(os.listdir(dataset_test_dir))
print(f"train_file_size: {train_file_size}\nval_file_size: {val_file_size}")

from smartpixels.DG.OptimizedDataGenerator_v2 import OptimizedDataGenerator
from smartpixels.losses.maxNLL import custom_loss
from smartpixels.models.conv2D import CreateModel


train_file_size: 80
val_file_size: 20


In [4]:
val_loader = OptimizedDataGenerator(
    load_from_tfrecords_dir= tfrecords_dir_val,
    shuffle=True,
    seed=SEED,
    quantize=False,
)

train_loader = OptimizedDataGenerator(
    load_from_tfrecords_dir = tfrecords_dir_train,
    shuffle=True,
    seed=SEED,
    quantize=False,
)


Loading metadata from /depot/cms/users/das214/datasets/dataset_3sr/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_val/metadata.json
Loading metadata from /depot/cms/users/das214/datasets/dataset_3sr/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/2t/TFR_train/metadata.json


In [5]:
model=CreateModel(shape = (16,16,2), 
                  n_filters=5, SepConv2D_kernel_size=3, 
                  Conv2D_kernel_size=1,
                  pool_size=3, 
                  hidden=16,
                  output = 14)

model.summary()

I0000 00:00:1752346865.385816 3052685 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38397 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB MIG 7g.40gb, pci bus id: 0000:21:00.0, compute capability: 8.0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 16, 16, 2)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d                │ (None, 14, 14, 5)      │            33 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 14, 14, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 14, 14, 5)      │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 14, 14, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 4, 4, 5)        │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 4, 4, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │         1,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 14)             │           238 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,869 (7.30 KB)

 Trainable params: 1,869 (7.30 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# model.compile(
#     optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),
#     loss=custom_loss,
# )
# history = model.fit(
#         x=train_loader,
#         validation_data=val_loader,
#         epochs=5,
#         shuffle=False,
#         verbose=1
#     )

## Add pruning and quantization
To add pruning and quantization, we need a config file that defines how to do that. Let's load a config file from `pquant/configs/`. <br/>
The training function we use later will add the pruning layers and quantized activations automatically using this config

In [7]:
from pquant import get_default_config
from IPython.display import JSON

# pruning_methods: "autosparse, cl, cs, dst, pdp, wanda, mdmm"
pruning_method = "mdmm"
config = get_default_config(pruning_method)
JSON(config)

<IPython.core.display.JSON object>

In [8]:
# Replace layers with compressed layers
from pquant import add_compression_layers
input_shape = (1, 16, 16, 2)

model = add_compression_layers(model, config, input_shape)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 16, 16, 2)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ compressed_layer_separable_con… │ (None, 14, 14, 5)      │            41 │
│ (CompressedLayerSeparableConv2… │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 14, 14, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ compressed_layer_conv2d_keras_1 │ (None, 14, 14, 5)      │            34 │
│ (CompressedLayerConv2dKeras)    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 14, 14, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 4, 4, 5)        │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 4, 4, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ compressed_layer_dense_keras    │ (None, 16)             │         1,300 │
│ (CompressedLayerDenseKeras)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ compressed_layer_dense_keras_1  │ (None, 16)             │           276 │
│ (CompressedLayerDenseKeras)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ compressed_layer_dense_keras_2  │ (None, 14)             │           242 │
│ (CompressedLayerDenseKeras)     │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,893 (7.39 KB)

 Trainable params: 1,869 (7.30 KB)

 Non-trainable params: 24 (96.00 B)

## Create data set
#### Let's create the data loader and the training and validation loops

In [9]:
from quantizers.fixed_point.fixed_point_ops import get_fixed_quantizer
# Set up input quantizer
quantizer = get_fixed_quantizer(overflow_mode="SAT")

from tqdm import tqdm
from pquant import get_layer_keep_ratio, get_model_losses
import re

def get_lambda_values(model):
    """Extracts lambda values with clean, descriptive keys for the progress bar."""
    lambda_info = {}
    # Use counters for each layer type to ensure unique, short keys
    counters = {'conv2d': 1, 'dense': 1, 'other': 1}

    for layer in model.layers:
        if "CompressedLayer" in layer.__class__.__name__ and hasattr(layer, 'pruning_layer'):
            pruning_layer = layer.pruning_layer
            if hasattr(pruning_layer, 'constraint_layer') and hasattr(pruning_layer.constraint_layer, 'lmbda'):
                layer_name = layer.name.lower()
                
                # Assign a clean, short key based on the layer type
                if 'conv2d' in layer_name:
                    key = f"λc2D{counters['conv2d']}"
                    counters['conv2d'] += 1
                elif 'dense' in layer_name:
                    key = f"λd{counters['dense']}"
                    counters['dense'] += 1
                else:
                    key = f"λo{counters['other']}"
                    counters['other'] += 1
                
                # Format the lambda value
                lambda_info[key] = f"{pruning_layer.constraint_layer.lmbda.numpy():.2f}"
                
    return lambda_info


@tf.function
def train_step(model, inputs, logits, optimizer: keras.optimizers.Optimizer):
    with tf.GradientTape() as tape:
        outputs = model(inputs, training=True)
        loss = custom_loss(logits, outputs)
        loss += get_model_losses(model, losses=keras.ops.convert_to_tensor(0.))
        if model.losses:
            loss += tf.add_n(model.losses)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss

@tf.function
def valid_step(model, inputs, logits):
    outputs = model(inputs, training=False)
    loss = custom_loss(logits, outputs)
    pquant_loss = get_model_losses(model, losses=keras.ops.convert_to_tensor(0.))
    if model.losses:
            loss += tf.add_n(model.losses)
    return loss + pquant_loss, pquant_loss

def train_smart_pixels_tf(model, epoch, trainloader, optimizer: keras.optimizers.Optimizer, *args, **kwargs):
    pbar = tqdm(range(len(trainloader)), desc=f"Epoch {epoch} [Training]")
    for batch_idx in pbar:
        inputs, logits = trainloader[batch_idx]
        loss = train_step(model, inputs, logits, optimizer)
        
        metrics = {
            'loss': f'{loss.numpy():.2f}',
        }
        metrics.update(get_lambda_values(model))
        pbar.set_postfix(**metrics)
        
    train_loader.on_epoch_end()

        

        
def validate_smart_pixels_tf(model, epoch, testloader, *args, **kwargs):
    total_loss = 0.0
    pbar = tqdm(range(len(testloader)), desc=f"Epoch {epoch} [Validation]")
    
    for batch_idx in pbar:
        inputs, logits = testloader[batch_idx]
        loss, pquant_loss = valid_step(model, inputs, logits)
        total_loss += loss
        pbar.set_postfix(loss=f'{loss.numpy():.2f}')

    testloader.on_epoch_end()
        
    avg_total_loss = total_loss / len(testloader)
    ratio = get_layer_keep_ratio(model)

    summary_string = (
        f"\tTotal Loss (NLL + MDMM): {avg_total_loss:.2f}\n"
        f"\tMDMM Loss:               {pquant_loss:.2f}\n"
        f"\tRemaining Weights:       {ratio * 100:.2f} %"
    )
    print(summary_string)


## Create loss function, scheduler and optimizer

## Train model
Training time. We use the train_compressed_model function from pquant to train. We need to provide some parameters such as training and validation functions, their input parameters, the model and the config file. The function automatically adds pruning layers and replaces activations with a quantized variant, trains the model, and removes the pruning layers after training is done

In [12]:
for layer in model.layers:
        if "CompressedLayer" in layer.__class__.__name__ and hasattr(layer, 'pruning_layer'):
            pruning_layer = layer.pruning_layer # This is your MDMM instance
            
            # Check if the MDMM instance has the constraint_layer
            if hasattr(pruning_layer, 'constraint_layer'):
                constraint = pruning_layer.constraint_layer
                
                # Finally, check for 'lmbda' on the constraint layer
                if hasattr(constraint, 'lmbda'):
                    print(f"  Layer: {layer.name}, Lambda Value: {constraint.lmbda.numpy():.4f}")


  Layer: compressed_layer_conv2d_keras_1, Lambda Value: 203.1964
  Layer: compressed_layer_dense_keras, Lambda Value: 130.0247
  Layer: compressed_layer_dense_keras_1, Lambda Value: 162.8332
  Layer: compressed_layer_dense_keras_2, Lambda Value: 102.0181


In [ ]:
from pathlib import Path
from secrets import token_hex

BASE_DIR = Path("/home/das214/PQuant/mdmm_dev/trainings").resolve()
RUN_DIR  = BASE_DIR / "smart_pixels"
FP       = token_hex(4)          

BASE_DIR.mkdir(parents=True, exist_ok=True)
RUN_DIR.mkdir(parents=True, exist_ok=True)

RUN_OUT = RUN_DIR / FP
RUN_OUT.mkdir(exist_ok=True)

print(f"Training artifacts will be stored in: {RUN_OUT}")

▶ Training artifacts will be stored in: /home/das214/PQuant/mdmm_dev/trainings/smart_pixels/cc963321


In [ ]:
from pquant.core.tf_impl.callbacks import CSVLogger, EarlyStopping 
from pquant.core.tf_impl.callbacks import ModelCheckpoint, EpochCheckpoint

cbs = [
    CSVLogger(RUN_OUT / "history.csv"),                   
    ModelCheckpoint(RUN_OUT / "best_weights.h5"),          
    EpochCheckpoint(RUN_OUT / "checkpoints"),              
    EarlyStopping(patience=10, min_delta=1e-3),            
]

In [11]:
from pquant import iterative_train
"""
Inputs to train_resnet we defined previously are:
          model, trainloader, device, loss_func, epoch, optimizer, scheduler, **kwargs
"""

optimizer =  keras.optimizers.Nadam(learning_rate=config["lr"])

trained_model = iterative_train(model = model, 
                                config = config, 
                                train_func = train_smart_pixels_tf, 
                                valid_func = validate_smart_pixels_tf, 
                                trainloader = train_loader, 
                                testloader = val_loader, 
                                device = None, 
                                loss_func = custom_loss,
                                optimizer = optimizer, 
                                )

Epoch 0 [Training]:   0%|          | 0/84 [00:00<?, ?it/s]2025-07-12 21:01:07.749129: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:381] TFRecordDataset `buffer_size` is unspecified, default to 262144
I0000 00:00:1752346870.181297 3053259 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1752346871.143700 3053259 cuda_solvers.cc:175] Creating GpuSolver handles for stream 0x5604cea575b0
Epoch 0 [Validation]: 100%|██████████| 21/21 [00:02<00:00,  7.36it/s, loss=16170.90]


	Total Loss (NLL + MDMM): 16036.61
	MDMM Loss:               4.96
	Remaining Weights:       99.61 %


Epoch 1 [Validation]: 100%|██████████| 21/21 [00:01<00:00, 14.94it/s, loss=8168.14]


	Total Loss (NLL + MDMM): 8091.20
	MDMM Loss:               8.90
	Remaining Weights:       99.61 %


Epoch 2 [Validation]: 100%|██████████| 21/21 [00:01<00:00, 14.52it/s, loss=3485.13]


...

	Total Loss (NLL + MDMM): -27024.05
	MDMM Loss:               3479.34
	Remaining Weights:       98.29 %


Epoch 715 [Validation]: 100%|██████████| 21/21 [00:01<00:00, 14.34it/s, loss=-27263.02]


	Total Loss (NLL + MDMM): -26905.73
	MDMM Loss:               3482.03
	Remaining Weights:       98.29 %


Epoch 716 [Training]:  62%|██████▏   | 52/84 [00:03<00:02, 13.17it/s, loss=-27055.28, λc2D1=203.20, λd1=130.02, λd2=162.83, λd3=102.02]


KeyboardInterrupt: 

In [ ]:
# TODO: 
#     - Increase number of epochs
#     - Implement early stopping
#     - Saving model weights (checkPoints)
#     - HLS4ml tutorial